In [1]:
print('l')

l


In [2]:
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
import os

import warnings
warnings.filterwarnings("ignore")

In [3]:
df_al=pd.read_csv("C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/#RAW/Data/All_Stations_Demographic.csv")

In [7]:
df_al=pd.read_excel("C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/#RAW/Data/FullSUPIdentifiedData.xlsx")

In [8]:
df_all=df_al[['stationid', 'Latitude', 'Longitude']]
df_all=df_all.drop_duplicates(subset=['stationid'], keep='first')
df_all.reset_index(drop=True, inplace=True)

In [9]:
gdf_all=gpd.GeoDataFrame(df_all, geometry=gpd.points_from_xy(df_all.Longitude, df_all.Latitude), crs='EPSG:4326')
gdf_all.to_crs(epsg=2277, inplace=True)

Landuse

### Preparation

#### Joining incomplete data

In [4]:
replica=gpd.read_file("C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/#RAW/Data/Replica data (land use)/Replica Texas Land Use Data/land-use_2021-Q4_Texas_created-2023-02-18/land-use_2021-Q4_Texas_created-2023-02-18.shp",
crs="EPSG:4326")
replica.to_crs(epsg=2277, inplace=True)

In [ ]:
gdf_all.columns

Index(['stationid', 'Latitude', 'Longitude', 'geometry'], dtype='object')

In [ ]:
gdf_all.head()

,stationid,Latitude,Longitude,geometry
0,MR0001,30.343975,-97.579744,POINT (3165091.572 10099589.793)
1,AU0116,30.410493,-97.711836,POINT (3122866.401 10122768.312)
2,AU0115,30.410030,-97.712725,POINT (3122590.214 10122593.372)
3,AU0114,30.405223,-97.725768,POINT (3118520.869 10120749.033)
4,AU0113,30.405455,-97.725944,POINT (3118463.424 10120832.085)


In [ ]:
# fig, ax=plt.subplots(1, figsize=(10,10))
# replica.plot(ax=ax)
# gdf_all[gdf_all.stationid=='PL0002'].plot(ax=ax, color='red')
# ax.set_ylim(11000000,11200000)
# ax.set_xlim(3200000,3600000)

In [ ]:
#Buffer
stations_buffer=gdf_all.copy(deep=True)
stations_buffer.geometry=stations_buffer.geometry.buffer(5280) #5280ft ~ 1mile

In [ ]:
stations_buffer['indexx']=stations_buffer.index
replica["area_parcel"]=replica.geometry.area
inter = gpd.overlay(replica,stations_buffer, how='intersection')
inter["area"]=inter.geometry.area

In [5]:
def get_buffer_info(inter):
    ws= lambda x: round(np.sum(x*(inter.loc[x.index, "area"])/(inter.loc[x.index, "area_parcel"])),5)
    f = {
        'la_total':ws, 'la_single':ws, 'la_multi':ws, 'la_retail':ws,
       'la_office':ws, 'la_attract':ws, 'la_mu_res':ws, 'la_mu_comm':ws, 'la_mu_indu':ws,
       'la_mu_oth':ws, 'la_industr':ws, 'la_health':ws, 'la_edu':ws, 'la_civ_oth':ws,
       'la_transpo':ws, 'la_space':ws, 'la_agri':ws, 'la_other':ws, 'la_unknown':ws,
        }

    newdf=inter.groupby("indexx").agg(f)
    return newdf

In [ ]:
newdf=get_buffer_info(inter)
DF1=pd.concat([newdf,stations_buffer[['stationid']]],axis=1)

In [29]:
buffer_radii = [ 2640]
buffer_radii_name = ['05mile']
#columns = ['stationid'] + [f'{roadway_type}_{buffer_radius}' for roadway_type, buffer_radius in itertools.product(roadway_types, buffer_radii_name )]
#df = pd.DataFrame(columns=columns)
DF=pd.DataFrame(index=range(681))
replica["area_parcel"]=replica.geometry.area
#Joining station and roadway network
for buffer_radius, buffer_radius_name in zip(buffer_radii, buffer_radii_name):
    stations_buffer = gdf_all.copy(deep=True)
    stations_buffer.geometry = gdf_all.geometry.buffer(buffer_radius)
    stations_buffer['indexx']=stations_buffer.index
    inter = gpd.overlay(replica,stations_buffer, how='intersection')
    inter["area"]=inter.geometry.area
    newdf=get_buffer_info(inter)
    DF1=pd.concat([newdf,stations_buffer[['stationid']]],axis=1)
    DF1['lu_replica_res'] = DF1[['la_single','la_multi','la_mu_res']].sum(axis=1)
    DF1['lu_replica_com'] = DF1[['la_retail','la_office','la_attract','la_mu_comm','la_health', 'la_edu', 'la_civ_oth', 'la_transpo']].sum(axis=1)
    DF1['lu_replica_ind'] = DF1[['la_mu_indu','la_industr','la_agri']].sum(axis=1)
    DF1['lu_replica_vac'] = DF1[['la_space','la_mu_oth','la_other', 'la_unknown']].sum(axis=1)
    DF1['sum_area'] = DF1['lu_replica_res'] + DF1['lu_replica_com'] + DF1['lu_replica_ind'] + DF1['lu_replica_vac']
    DF1_=(DF1[['lu_replica_res','lu_replica_com','lu_replica_ind','lu_replica_vac']]).div(DF1['sum_area'], axis=0)
    DF1=DF1[['stationid','lu_replica_res','lu_replica_com','lu_replica_ind','lu_replica_vac']]
    DF1['lu_replica_entropy']=-1*(np.log(DF1_)*DF1_).sum(axis=1)/np.log(4)
    DF1.rename(columns={'lu_replica_res':'lu_replica_res_'+buffer_radius_name,
                       'lu_replica_com':'lu_replica_com_'+buffer_radius_name,
                       'lu_replica_ind':'lu_replica_ind_'+buffer_radius_name,
                       'lu_replica_vac':'lu_replica_vac_'+buffer_radius_name,
                       'lu_replica_entropy':'lu_replica_entropy_'+buffer_radius_name}, inplace=True)
    DF=pd.concat([DF,DF1],axis=1)

In [30]:
DF.lu_replica_entropy_05mile.mean()

0.7095151637968277

In [31]:
DF.lu_replica_entropy_05mile.max()

0.9894745118450083

In [32]:
DF.lu_replica_entropy_05mile.min()

0.14839127301101768

In [33]:
DF.lu_replica_entropy_05mile.std()

0.14605235331404004

In [34]:
DF.to_csv('Data/Temp/FULL_replicalanduse_v3.csv', index=False)